In [4]:
#basic packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LogisticRegression    
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.decomposition import PCA

import time

from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from xgboost import XGBClassifier
from sklearn.ensemble import AdaBoostClassifier

In [5]:
df = pd.read_csv('data/DJIA_table.csv')
dfCombined = pd.read_csv('data/Combined_News_DJIA_Prices.csv')
dfCombined.describe()

,Label,Open,High,Low,Close,Volume,Adj Close
count,1989.000000,1989.000000,1989.000000,1989.000000,1989.000000,1.989000e+03,1989.000000
mean,0.535445,13459.116049,13541.303173,13372.931728,13463.032255,1.628110e+08,13463.032255
std,0.498867,3143.281634,3136.271725,3150.420934,3144.006996,9.392343e+07,3144.006996
min,0.000000,6547.009766,6709.609863,6469.950195,6547.049805,8.410000e+06,6547.049805
25%,0.000000,10907.339840,11000.980470,10824.759770,10913.379880,1.000000e+08,10913.379880
50%,1.000000,13022.049810,13088.110350,12953.129880,13025.580080,1.351700e+08,13025.580080
75%,1.000000,16477.699220,16550.070310,16392.769530,16478.410160,1.926000e+08,16478.410160
max,1.000000,18315.060550,18351.359380,18272.560550,18312.390630,6.749200e+08,18312.390630


In [6]:
dfCombined[['Date','Label','Close']].head()

,Date,Label,Close
0,08-08-2008,0,11734.32031
1,11-08-2008,1,11782.34961
2,12-08-2008,0,11642.46973
3,13-08-2008,0,11532.95996
4,14-08-2008,1,11615.92969


In [7]:
def create_dataset(x_in, y_in, window=10 ):
    x = []
    y = []

    #print("x",x_in.shape,"y : ", y_in.shape[0])
    for i in range(window, x_in.shape[0]):
           x.append(x_in[i-window:i].flatten())
           y.append(y_in[i])  

    x = np.array(x)    
    y = np.array(y)    
    
    #print(x.shape, y.shape)
    return x,y

In [8]:
def save_results_to_csv(results, colnames, file_name_suffix ):
    results_df = pd.DataFrame(results,columns=colnames)
    fname=file_name_suffix+str(time.time())+'.csv'
    results_df.to_csv(fname)

In [9]:
prices= dfCombined['Close'].values
y_arr = np.array(dfCombined['Label'])

In [19]:
attribsSet = [['Close'],['Close','Volume'],['Close','Volume','High','Low']]
resultsList=['Attributes','Train Per','Window size','Classifer','Using PCA','Accuracy']
train_percentages=[0.1,0.2,0.3,0.5,0.6,0.8]
#train_percentages=[0.2,0.5,0.7]
window_sizes = [3,5,10,15,20,25,30,40,50,55,60,63,65,70,80,90,100,110,120,130,150]
#window_sizes = range(5,205,5)
modelList = []
modelList.append(("LogisticReg",LogisticRegression(random_state=42)))
modelList.append(("RandomForestClassifier",RandomForestClassifier(random_state=42)))
modelList.append(("SVC",SVC(random_state=42)))
modelList.append(("DecisionTreeClassifier",DecisionTreeClassifier(random_state=42)))
modelList.append(("KNeighborsClassifier",KNeighborsClassifier(10)))
modelList.append(("GaussianNB",GaussianNB()))
modelList.append(("XGBClassifier",XGBClassifier()))
modelList.append(("AdaBoostClassifier",AdaBoostClassifier()))    
ts_acc_best=-2.0
best=[]    
#attribsSet = [['Close','Volume','High','Low']]
for attribIndex in range(0,len(attribsSet)):    
    closeVoldf = dfCombined[attribsSet[attribIndex]].values                                        
    y_arr = np.array(dfCombined['Label'])
    for train_per in train_percentages:            
        for window in window_sizes:
            #print(dataset_train.shape)
            #print(dataset_test.shape)


            dataset_train = np.array(closeVoldf[:int(closeVoldf.shape[0]*train_per)])
            y_train = np.array(y_arr[:int(y_arr.shape[0]*train_per)])
            dataset_test = np.array(closeVoldf[int(closeVoldf.shape[0]*train_per)-window:])
            y_test = np.array(y_arr[int(y_arr.shape[0]*train_per)-window:])

            min_max_scaler = MinMaxScaler(feature_range=(0,1))
            dataset_train = min_max_scaler.fit_transform(dataset_train)
            dataset_test = min_max_scaler.transform(dataset_test)

            #print(dataset_train.shape,y_train.shape)
            #print(dataset_test.shape,y_test.shape)

            x_train, y_train = create_dataset(dataset_train,y_train,window)
            x_test, y_test   = create_dataset(dataset_test,y_test,window)

           
            for name, classifier in modelList:                                      
                classifier.fit(x_train,y_train)
                ytr_pred = classifier.predict(x_train)
                yts_pred = classifier.predict(x_test)

                tr_acc = accuracy_score(y_train,ytr_pred)
                ts_acc= accuracy_score(y_test,yts_pred)
               # data[name] = [attribsSet[attribIndex],tr_acc,ts_acc]
                print('Completed ',train_per,window,name,attribsSet[attribIndex]," Score[test]",ts_acc)
                resultsList.append([attribsSet[attribIndex],train_per,window,name,'N',ts_acc])
                if(ts_acc_best <= ts_acc) :
                    ts_acc_best = ts_acc
                    best = [train_per,window,name,attribsSet[attribIndex],'No PCA']
                    
                pca_final = PCA(0.90)
                df_train_pca = pca_final.fit_transform(x_train)
                df_test_pca = pca_final.transform(x_test)
                
                classifier.fit(df_train_pca,y_train)
                ytr_pred_pca = classifier.predict(df_train_pca)
                yts_pred_pca = classifier.predict(df_test_pca)
                tr_acc_pca = accuracy_score(y_train,ytr_pred_pca)
                ts_acc_pca = accuracy_score(y_test,yts_pred_pca)
                
                resultsList.append([attribsSet[attribIndex],train_per,window,name,'Y',ts_acc_pca])
                print('Completed with PCA with ',pca_final.n_components_,' components',train_per,window,name,attribsSet[attribIndex]," Score[test]",ts_acc_pca)
                if(ts_acc_best <= ts_acc_pca) :
                    ts_acc_best = ts_acc_pca
                    best = [train_per,window,name,attribsSet[attribIndex],'With PCA']
                
            
            
            


Completed  0.1 3 LogisticReg ['Close']  Score[test] 0.4567280848687884
Completed with PCA with  1  components 0.1 3 LogisticReg ['Close']  Score[test] 0.4567280848687884
Completed  0.1 3 RandomForestClassifier ['Close']  Score[test] 0.46789503070910105
Completed with PCA with  1  components 0.1 3 RandomForestClassifier ['Close']  Score[test] 0.4723618090452261
Completed  0.1 3 SVC ['Close']  Score[test] 0.5120044667783361
Completed with PCA with  1  components 0.1 3 SVC ['Close']  Score[test] 0.5080960357342267
Completed  0.1 3 DecisionTreeClassifier ['Close']  Score[test] 0.46901172529313234
Completed with PCA with  1  components 0.1 3 DecisionTreeClassifier ['Close']  Score[test] 0.4723618090452261
Completed  0.1 3 KNeighborsClassifier ['Close']  Score[test] 0.45784477945281965
Completed with PCA with  1  components 0.1 3 KNeighborsClassifier ['Close']  Score[test] 0.4584031267448353
Completed  0.1 3 GaussianNB ['Close']  Score[test] 0.5159128978224455
Completed with PCA with  1  com

Completed  0.1 25 XGBClassifier ['Close']  Score[test] 0.5053042992741485
Completed with PCA with  1  components 0.1 25 XGBClassifier ['Close']  Score[test] 0.52428810720268
Completed  0.1 25 AdaBoostClassifier ['Close']  Score[test] 0.5064209938581797
Completed with PCA with  1  components 0.1 25 AdaBoostClassifier ['Close']  Score[test] 0.4567280848687884
Completed  0.1 30 LogisticReg ['Close']  Score[test] 0.4567280848687884
Completed with PCA with  2  components 0.1 30 LogisticReg ['Close']  Score[test] 0.4567280848687884
Completed  0.1 30 RandomForestClassifier ['Close']  Score[test] 0.4997208263539922
Completed with PCA with  2  components 0.1 30 RandomForestClassifier ['Close']  Score[test] 0.45784477945281965
Completed  0.1 30 SVC ['Close']  Score[test] 0.4567280848687884
Completed with PCA with  2  components 0.1 30 SVC ['Close']  Score[test] 0.4567280848687884
Completed  0.1 30 DecisionTreeClassifier ['Close']  Score[test] 0.5019542155220548
Completed with PCA with  2  compon

Completed  0.1 63 XGBClassifier ['Close']  Score[test] 0.4567280848687884
Completed with PCA with  4  components 0.1 63 XGBClassifier ['Close']  Score[test] 0.4617532104969291
Completed  0.1 63 AdaBoostClassifier ['Close']  Score[test] 0.4567280848687884
Completed with PCA with  4  components 0.1 63 AdaBoostClassifier ['Close']  Score[test] 0.45616973757677276
Completed  0.1 65 LogisticReg ['Close']  Score[test] 0.4567280848687884
Completed with PCA with  4  components 0.1 65 LogisticReg ['Close']  Score[test] 0.4567280848687884
Completed  0.1 65 RandomForestClassifier ['Close']  Score[test] 0.4567280848687884
Completed with PCA with  4  components 0.1 65 RandomForestClassifier ['Close']  Score[test] 0.46621998883305416
Completed  0.1 65 SVC ['Close']  Score[test] 0.4567280848687884
Completed with PCA with  4  components 0.1 65 SVC ['Close']  Score[test] 0.4567280848687884
Completed  0.1 65 DecisionTreeClassifier ['Close']  Score[test] 0.45896147403685095
Completed with PCA with  4  co

Completed  0.1 110 XGBClassifier ['Close']  Score[test] 0.46119486320491343
Completed with PCA with  6  components 0.1 110 XGBClassifier ['Close']  Score[test] 0.47794528196538244
Completed  0.1 110 AdaBoostClassifier ['Close']  Score[test] 0.45951982132886654
Completed with PCA with  6  components 0.1 110 AdaBoostClassifier ['Close']  Score[test] 0.4796203238414294
Completed  0.1 120 LogisticReg ['Close']  Score[test] 0.4567280848687884
Completed with PCA with  7  components 0.1 120 LogisticReg ['Close']  Score[test] 0.4584031267448353
Completed  0.1 120 RandomForestClassifier ['Close']  Score[test] 0.4952540480178671
Completed with PCA with  7  components 0.1 120 RandomForestClassifier ['Close']  Score[test] 0.45784477945281965
Completed  0.1 120 SVC ['Close']  Score[test] 0.4567280848687884
Completed with PCA with  7  components 0.1 120 SVC ['Close']  Score[test] 0.4567280848687884
Completed  0.1 120 DecisionTreeClassifier ['Close']  Score[test] 0.5438302624232273
Completed with PCA

Completed  0.2 10 GaussianNB ['Close']  Score[test] 0.5383165829145728
Completed with PCA with  1  components 0.2 10 GaussianNB ['Close']  Score[test] 0.5383165829145728
Completed  0.2 10 XGBClassifier ['Close']  Score[test] 0.46545226130653267
Completed with PCA with  1  components 0.2 10 XGBClassifier ['Close']  Score[test] 0.4667085427135678
Completed  0.2 10 AdaBoostClassifier ['Close']  Score[test] 0.4704773869346734
Completed with PCA with  1  components 0.2 10 AdaBoostClassifier ['Close']  Score[test] 0.5182160804020101
Completed  0.2 15 LogisticReg ['Close']  Score[test] 0.5062814070351759
Completed with PCA with  1  components 0.2 15 LogisticReg ['Close']  Score[test] 0.5062814070351759
Completed  0.2 15 RandomForestClassifier ['Close']  Score[test] 0.47173366834170855
Completed with PCA with  1  components 0.2 15 RandomForestClassifier ['Close']  Score[test] 0.46608040201005024
Completed  0.2 15 SVC ['Close']  Score[test] 0.47361809045226133
Completed with PCA with  1  compon

...

Completed  0.8 120 LogisticReg ['Close', 'Volume', 'High', 'Low']  Score[test] 0.4748743718592965
Completed with PCA with  1  components 0.8 120 LogisticReg ['Close', 'Volume', 'High', 'Low']  Score[test] 0.507537688442211
Completed  0.8 120 RandomForestClassifier ['Close', 'Volume', 'High', 'Low']  Score[test] 0.457286432160804
Completed with PCA with  1  components 0.8 120 RandomForestClassifier ['Close', 'Volume', 'High', 'Low']  Score[test] 0.48743718592964824
Completed  0.8 120 SVC ['Close', 'Volume', 'High', 'Low']  Score[test] 0.507537688442211
Completed with PCA with  1  components 0.8 120 SVC ['Close', 'Volume', 'High', 'Low']  Score[test] 0.507537688442211
Completed  0.8 120 DecisionTreeClassifier ['Close', 'Volume', 'High', 'Low']  Score[test] 0.5125628140703518
Completed with PCA with  1  components 0.8 120 DecisionTreeClassifier ['Close', 'Volume', 'High', 'Low']  Score[test] 0.48743718592964824
Completed  0.8 120 KNeighborsClassifier ['Close', 'Volume', 'High', 'Low']  Sc

C:\Users\anagh\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Completed  0.8 130 LogisticReg ['Close', 'Volume', 'High', 'Low']  Score[test] 0.4798994974874372
Completed with PCA with  1  components 0.8 130 LogisticReg ['Close', 'Volume', 'High', 'Low']  Score[test] 0.507537688442211
Completed  0.8 130 RandomForestClassifier ['Close', 'Volume', 'High', 'Low']  Score[test] 0.4723618090452261
Completed with PCA with  1  components 0.8 130 RandomForestClassifier ['Close', 'Volume', 'High', 'Low']  Score[test] 0.5125628140703518
Completed  0.8 130 SVC ['Close', 'Volume', 'High', 'Low']  Score[test] 0.507537688442211
Completed with PCA with  1  components 0.8 130 SVC ['Close', 'Volume', 'High', 'Low']  Score[test] 0.507537688442211
Completed  0.8 130 DecisionTreeClassifier ['Close', 'Volume', 'High', 'Low']  Score[test] 0.5326633165829145
Completed with PCA with  1  components 0.8 130 DecisionTreeClassifier ['Close', 'Volume', 'High', 'Low']  Score[test] 0.5125628140703518
Completed  0.8 130 KNeighborsClassifier ['Close', 'Volume', 'High', 'Low']  Sco

C:\Users\anagh\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Completed with PCA with  1  components 0.8 150 LogisticReg ['Close', 'Volume', 'High', 'Low']  Score[test] 0.507537688442211
Completed  0.8 150 RandomForestClassifier ['Close', 'Volume', 'High', 'Low']  Score[test] 0.4798994974874372
Completed with PCA with  1  components 0.8 150 RandomForestClassifier ['Close', 'Volume', 'High', 'Low']  Score[test] 0.5025125628140703
Completed  0.8 150 SVC ['Close', 'Volume', 'High', 'Low']  Score[test] 0.507537688442211
Completed with PCA with  1  components 0.8 150 SVC ['Close', 'Volume', 'High', 'Low']  Score[test] 0.507537688442211
Completed  0.8 150 DecisionTreeClassifier ['Close', 'Volume', 'High', 'Low']  Score[test] 0.4748743718592965
Completed with PCA with  1  components 0.8 150 DecisionTreeClassifier ['Close', 'Volume', 'High', 'Low']  Score[test] 0.5025125628140703
Completed  0.8 150 KNeighborsClassifier ['Close', 'Volume', 'High', 'Low']  Score[test] 0.4396984924623116
Completed with PCA with  1  components 0.8 150 KNeighborsClassifier ['

In [20]:
save_results_to_csv(resultsList[6:],['Attributes','Train Per','Window size','Classifer','Using PCA','Accuracy'],'results_classification')
print("Best: ",best, "Accuracy ",ts_acc_best)        

Best:  [0.8, 3, 'AdaBoostClassifier', ['Close'], 'No PCA'] Accuracy  0.5653266331658291


In [45]:
#creating samples from the testing/training dataset
def create_dataset_reg(x_in, price, label, window=10):
    x = []
    y = []
    z = []
    
    for i in range(window, x_in.shape[0]):        
        x.append(x_in[i-window:i].flatten())
        y.append(price[i])
        z.append(label[i])  
    x = np.array(x)
    y = np.array(y)
    z = np.array(z)
    return x,y,z

In [46]:
def direction(price_tplusone, price_t):
    if price_tplusone > price_t :
        return 1
    return 0

In [47]:
def accuracy_of_direction(y_predicted, label_test,price_test):                
                label_predicted = []            
                label_predicted.append(direction(y_predicted[0],price_test[0]))
                for index in range(1,len(y_predicted)):
                     label_predicted.append(direction(y_predicted[index], y_predicted[index-1]))
                    
                
                ts_acc_pca = accuracy_score(label_test,label_predicted)
                return ts_acc_pca

In [48]:
attribsSet = [['Close'],['Close','Volume'],['Close','Volume','High','Low']]
#attribsSet = [['Close','Volume','High','Low']]
ts_acc_best=-2.0
best=[]
resultsReg=[]
for attribIndex in range(0,len(attribsSet)):    
    closeVoldf = dfCombined[attribsSet[attribIndex]].values                                
    train_percentages=[0.1,0.2,0.3,0.5,0.6,0.8]
    #train_percentages=[0.2,0.5,0.7]
    window_sizes = [3,5,10,15,20,25,30,40,50,55,60,63,65,70,80,90,100,110,120,130,150]
    modelList = []
    modelList.append(("LinearRegression",LinearRegression()))
        
    ts_acc_best=-2.0
    best=[]
    results={}    
    price_arr = np.array(dfCombined['Close'])
    z_arr = np.array(dfCombined['Label'])
    
    for train_per in train_percentages:            
        for window in window_sizes:
            
            dataset_train = np.array(closeVoldf[:int(closeVoldf.shape[0]*train_per)])
            price_arr_train = np.array(price_arr[:int(price_arr.shape[0]*train_per)])
            z_arr_train = np.array(z_arr[:int(z_arr.shape[0]*train_per)])
            
            dataset_test = np.array(closeVoldf[int(closeVoldf.shape[0]*train_per)-window:])
            price_arr_test = np.array(price_arr[int(price_arr.shape[0]*train_per)-window:])
            z_arr_test = np.array(z_arr[int(z_arr.shape[0]*train_per)-window:])
            
            price_arr_train = price_arr_train.reshape(-1,1)
            price_arr_test = price_arr_test.reshape(-1,1)

            min_max_scaler = MinMaxScaler(feature_range=(0,1))
            dataset_train = min_max_scaler.fit_transform(dataset_train)
            price_arr_train = MinMaxScaler(feature_range=(0,1)).fit_transform(price_arr_train)
            dataset_test = min_max_scaler.fit_transform(dataset_test)
            price_arr_test = MinMaxScaler(feature_range=(0,1)).fit_transform(price_arr_test)
            
            
            x_train, price_train, label_train = create_dataset_reg(dataset_train, price_arr_train, z_arr_train,window)
            x_test, price_test, label_test = create_dataset_reg(dataset_test, price_arr_test, z_arr_test,window)
            
            
            for name, regression in modelList: 
                
                regression.fit(x_train,price_train)            
                y_pred_test = regression.predict(x_test)                                
                ts_acc = accuracy_of_direction(y_pred_test,label_test,price_test)                
                resultsReg.append([attribsSet[attribIndex],train_per,window,name,'N',ts_acc])
                if(ts_acc_best <= ts_acc) :
                        ts_acc_best = ts_acc
                        best = [train_per,window,attribsSet[attribIndex],'No PCA']

                        
                pca_final = PCA(0.95)
                df_train_pca = pca_final.fit_transform(x_train)
                df_test_pca = pca_final.transform(x_test)
                regression.fit(df_train_pca,price_train)            
                y_pred_test_pca = regression.predict(df_test_pca)           
                print(y_pred_test_pca.shape,price_test.shape)
                ts_acc_pca = accuracy_of_direction(y_pred_test_pca,label_test,price_test)
                resultsReg.append([attribsSet[attribIndex],train_per,window,name,'Y',ts_acc_pca])

                if(ts_acc_best <= ts_acc_pca) :
                        ts_acc_best = ts_acc_pca
                        best = [train_per,window,attribsSet[attribIndex],'No PCA']
                            
                print('Completed ',name,attribsSet[attribIndex],train_per,window,'Acc ',ts_acc,'Acc PCA components ',pca_final.n_components_,ts_acc_pca)


#print(resultsReg)

(1791, 1) (1791, 1)
Completed  LinearRegression ['Close'] 0.1 3 Acc  0.48576214405360135 Acc PCA components  1 0.5069793411501954
(1791, 1) (1791, 1)
Completed  LinearRegression ['Close'] 0.1 5 Acc  0.48911222780569513 Acc PCA components  1 0.507537688442211
(1791, 1) (1791, 1)
Completed  LinearRegression ['Close'] 0.1 10 Acc  0.47571189279731996 Acc PCA components  1 0.5108877721943048
(1791, 1) (1791, 1)
Completed  LinearRegression ['Close'] 0.1 15 Acc  0.47738693467336685 Acc PCA components  2 0.5103294249022893
(1791, 1) (1791, 1)
Completed  LinearRegression ['Close'] 0.1 20 Acc  0.4723618090452261 Acc PCA components  2 0.5097710776102736
(1791, 1) (1791, 1)
Completed  LinearRegression ['Close'] 0.1 25 Acc  0.47627024008933555 Acc PCA components  2 0.5108877721943048
(1791, 1) (1791, 1)
Completed  LinearRegression ['Close'] 0.1 30 Acc  0.480178671133445 Acc PCA components  3 0.5131211613623674
(1791, 1) (1791, 1)
Completed  LinearRegression ['Close'] 0.1 40 Acc  0.4952540480178671 

Completed  LinearRegression ['Close'] 0.5 20 Acc  0.4743718592964824 Acc PCA components  1 0.4994974874371859
(995, 1) (995, 1)
Completed  LinearRegression ['Close'] 0.5 25 Acc  0.4763819095477387 Acc PCA components  1 0.4904522613065327
(995, 1) (995, 1)
Completed  LinearRegression ['Close'] 0.5 30 Acc  0.471356783919598 Acc PCA components  1 0.5005025125628141
(995, 1) (995, 1)
Completed  LinearRegression ['Close'] 0.5 40 Acc  0.4663316582914573 Acc PCA components  1 0.5045226130653266
(995, 1) (995, 1)
Completed  LinearRegression ['Close'] 0.5 50 Acc  0.47738693467336685 Acc PCA components  2 0.5045226130653266
(995, 1) (995, 1)
Completed  LinearRegression ['Close'] 0.5 55 Acc  0.46934673366834173 Acc PCA components  2 0.5065326633165829
(995, 1) (995, 1)
Completed  LinearRegression ['Close'] 0.5 60 Acc  0.4844221105527638 Acc PCA components  2 0.4984924623115578
(995, 1) (995, 1)
Completed  LinearRegression ['Close'] 0.5 63 Acc  0.4884422110552764 Acc PCA components  2 0.4894472361

(1791, 1) (1791, 1)
Completed  LinearRegression ['Close', 'Volume'] 0.1 30 Acc  0.5025125628140703 Acc PCA components  25 0.5058626465661642
(1791, 1) (1791, 1)
Completed  LinearRegression ['Close', 'Volume'] 0.1 40 Acc  0.5013958682300391 Acc PCA components  32 0.49246231155778897
(1791, 1) (1791, 1)
Completed  LinearRegression ['Close', 'Volume'] 0.1 50 Acc  0.5025125628140703 Acc PCA components  38 0.49022892238972643
(1791, 1) (1791, 1)
Completed  LinearRegression ['Close', 'Volume'] 0.1 55 Acc  0.4952540480178671 Acc PCA components  41 0.4919039642657733
(1791, 1) (1791, 1)
Completed  LinearRegression ['Close', 'Volume'] 0.1 60 Acc  0.4991624790619765 Acc PCA components  44 0.5025125628140703
(1791, 1) (1791, 1)
Completed  LinearRegression ['Close', 'Volume'] 0.1 63 Acc  0.5019542155220548 Acc PCA components  45 0.4946957007258515
(1791, 1) (1791, 1)
Completed  LinearRegression ['Close', 'Volume'] 0.1 65 Acc  0.4896705750977108 Acc PCA components  46 0.5064209938581797
(1791, 1) (

(995, 1) (995, 1)
Completed  LinearRegression ['Close', 'Volume'] 0.5 25 Acc  0.4814070351758794 Acc PCA components  14 0.5015075376884423
(995, 1) (995, 1)
Completed  LinearRegression ['Close', 'Volume'] 0.5 30 Acc  0.4733668341708543 Acc PCA components  16 0.4964824120603015
(995, 1) (995, 1)
Completed  LinearRegression ['Close', 'Volume'] 0.5 40 Acc  0.4733668341708543 Acc PCA components  21 0.4984924623115578
(995, 1) (995, 1)
Completed  LinearRegression ['Close', 'Volume'] 0.5 50 Acc  0.4844221105527638 Acc PCA components  25 0.5005025125628141
(995, 1) (995, 1)
Completed  LinearRegression ['Close', 'Volume'] 0.5 55 Acc  0.47537688442211057 Acc PCA components  27 0.4964824120603015
(995, 1) (995, 1)
Completed  LinearRegression ['Close', 'Volume'] 0.5 60 Acc  0.4894472361809045 Acc PCA components  29 0.49447236180904525
(995, 1) (995, 1)
Completed  LinearRegression ['Close', 'Volume'] 0.5 63 Acc  0.4834170854271357 Acc PCA components  30 0.4884422110552764
(995, 1) (995, 1)
Complet

Completed  LinearRegression ['Close', 'Volume', 'High', 'Low'] 0.1 10 Acc  0.4980457844779453 Acc PCA components  6 0.4980457844779453
(1791, 1) (1791, 1)
Completed  LinearRegression ['Close', 'Volume', 'High', 'Low'] 0.1 15 Acc  0.4952540480178671 Acc PCA components  9 0.49022892238972643
(1791, 1) (1791, 1)
Completed  LinearRegression ['Close', 'Volume', 'High', 'Low'] 0.1 20 Acc  0.4980457844779453 Acc PCA components  12 0.49860413176996093
(1791, 1) (1791, 1)
Completed  LinearRegression ['Close', 'Volume', 'High', 'Low'] 0.1 25 Acc  0.5097710776102736 Acc PCA components  16 0.49581239530988275
(1791, 1) (1791, 1)
Completed  LinearRegression ['Close', 'Volume', 'High', 'Low'] 0.1 30 Acc  0.5114461194863205 Acc PCA components  19 0.5019542155220548
(1791, 1) (1791, 1)
Completed  LinearRegression ['Close', 'Volume', 'High', 'Low'] 0.1 40 Acc  0.49022892238972643 Acc PCA components  25 0.48911222780569513
(1791, 1) (1791, 1)
Completed  LinearRegression ['Close', 'Volume', 'High', 'Low'

(1393, 1) (1393, 1)
Completed  LinearRegression ['Close', 'Volume', 'High', 'Low'] 0.3 70 Acc  0.4745154343144293 Acc PCA components  25 0.5125628140703518
(1393, 1) (1393, 1)
Completed  LinearRegression ['Close', 'Volume', 'High', 'Low'] 0.3 80 Acc  0.4781048097631012 Acc PCA components  28 0.5039483129935391
(1393, 1) (1393, 1)
Completed  LinearRegression ['Close', 'Volume', 'High', 'Low'] 0.3 90 Acc  0.48887293610911703 Acc PCA components  30 0.5082555635319455
(1393, 1) (1393, 1)
Completed  LinearRegression ['Close', 'Volume', 'High', 'Low'] 0.3 100 Acc  0.4809763101220388 Acc PCA components  33 0.5175879396984925
(1393, 1) (1393, 1)
Completed  LinearRegression ['Close', 'Volume', 'High', 'Low'] 0.3 110 Acc  0.4781048097631012 Acc PCA components  35 0.49246231155778897
(1393, 1) (1393, 1)
Completed  LinearRegression ['Close', 'Volume', 'High', 'Low'] 0.3 120 Acc  0.47595118449389806 Acc PCA components  37 0.5010768126346016
(1393, 1) (1393, 1)
Completed  LinearRegression ['Close', 

(398, 1) (398, 1)
Completed  LinearRegression ['Close', 'Volume', 'High', 'Low'] 0.8 25 Acc  0.4723618090452261 Acc PCA components  2 0.4899497487437186
(398, 1) (398, 1)
Completed  LinearRegression ['Close', 'Volume', 'High', 'Low'] 0.8 30 Acc  0.4748743718592965 Acc PCA components  3 0.45226130653266333
(398, 1) (398, 1)
Completed  LinearRegression ['Close', 'Volume', 'High', 'Low'] 0.8 40 Acc  0.47738693467336685 Acc PCA components  3 0.46984924623115576
(398, 1) (398, 1)
Completed  LinearRegression ['Close', 'Volume', 'High', 'Low'] 0.8 50 Acc  0.5 Acc PCA components  3 0.4748743718592965
(398, 1) (398, 1)
Completed  LinearRegression ['Close', 'Volume', 'High', 'Low'] 0.8 55 Acc  0.4849246231155779 Acc PCA components  3 0.47738693467336685
(398, 1) (398, 1)
Completed  LinearRegression ['Close', 'Volume', 'High', 'Low'] 0.8 60 Acc  0.4949748743718593 Acc PCA components  3 0.4748743718592965
(398, 1) (398, 1)
Completed  LinearRegression ['Close', 'Volume', 'High', 'Low'] 0.8 63 Acc  

In [49]:
save_results_to_csv(resultsReg,
                    ['Attributes','Train Per','Window size','Classifer','Using PCA','Accuracy']
                    ,'results_regression')
                
print('Best ',best,'Acc',ts_acc_best)  

Best  [0.5, 130, ['Close', 'Volume', 'High', 'Low'], 'No PCA'] Acc 0.5256281407035176


In [38]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [39]:
def create_models(window):  
            result=[]
            ann_name='model_1_2hidden_relu_adam'
            #building the model
            model = Sequential()
            model.add(Dense(window, activation='relu', input_shape=(window*len(attribsSet[attribIndex]),)))
            model.add(Dense(int(window*1.5), activation='relu'))
            model.add(Dense(int(window*1.5), activation='relu'))                            
            model.add(Dense(output_size, activation='sigmoid'))            
            model.compile(optimizer='adam', loss='mean_squared_error')
            result.append([ann_name,model])
            
            ann_name='model_2_2hidden_tanh_adam'
            #building the model
            model = Sequential()
            model.add(Dense(window, activation='tanh', input_shape=(window*len(attribsSet[attribIndex]),)))
            model.add(Dense(int(window*1.5), activation='tanh'))
            model.add(Dense(int(window*1.5), activation='tanh'))                            
            model.add(Dense(output_size, activation='tanh'))            
            model.compile(optimizer='adam', loss='mean_squared_error')
            result.append([ann_name,model])
            
            ann_name='model_3_2hidden_softmax_adam'
            #building the model
            model = Sequential()
            model.add(Dense(window, activation='softmax', input_shape=(window*len(attribsSet[attribIndex]),)))
            model.add(Dense(int(window*1.5), activation='softmax'))
            model.add(Dense(int(window*1.5), activation='softmax'))                            
            model.add(Dense(output_size, activation='softmax'))            
            model.compile(optimizer='adam', loss='mean_squared_error')
            result.append([ann_name,model])
            
            ann_name='model_4_2hidden_softmax_sgd'
            #building the model
            model = Sequential()
            model.add(Dense(window, activation='softmax', input_shape=(window*len(attribsSet[attribIndex]),)))
            model.add(Dense(int(window*1.5), activation='softmax'))
            model.add(Dense(int(window*1.5), activation='softmax'))                            
            model.add(Dense(output_size, activation='softmax'))            
            model.compile(optimizer='sgd', loss='mean_squared_error')
            result.append([ann_name,model])
            
            ann_name='model_5_2hidden_softmax_adagrad'
            #building the model
            model = Sequential()
            model.add(Dense(window, activation='softmax', input_shape=(window*len(attribsSet[attribIndex]),)))
            model.add(Dense(int(window*1.5), activation='softmax'))
            model.add(Dense(int(window*1.5), activation='softmax'))                            
            model.add(Dense(output_size, activation='softmax'))                        
            model.compile(optimizer='adagrad', loss='mean_squared_error')
            result.append([ann_name,model])
            
             
            
            ann_name='model_6_4hidden_relu_adam'
            #building the model
            model = Sequential()
            model.add(Dense(window, activation='relu', input_shape=(window*len(attribsSet[attribIndex]),)))
            model.add(Dense(int(window*1.5), activation='relu'))
            model.add(Dense(int(window*2), activation='relu'))                            
            model.add(Dense(int(window*2), activation='relu'))  
            model.add(Dense(int(window*1.5), activation='relu'))
            model.add(Dense(output_size, activation='relu'))            
            model.compile(optimizer='adam', loss='mean_squared_error')
            result.append([ann_name,model])
            
            
            
            ann_name='model_7_4hidden_tanh_relu_adam'
            #building the model
            model = Sequential()
            model.add(Dense(window, activation='tanh', input_shape=(window*len(attribsSet[attribIndex]),)))
            model.add(Dense(int(window*1.5), activation='relu'))
            model.add(Dense(int(window*2), activation='relu'))                            
            model.add(Dense(int(window*2), activation='tanh'))  
            model.add(Dense(int(window*1.5), activation='tanh'))
            model.add(Dense(output_size, activation='relu'))            
            model.compile(optimizer='adam', loss='mean_squared_error')
            result.append([ann_name,model])
            
            
            ann_name='model_8_4hidden_tanh_softmax_adagrad'
            #building the model
            model = Sequential()
            model.add(Dense(window, activation='tanh', input_shape=(window*len(attribsSet[attribIndex]),)))
            model.add(Dense(int(window*1.5), activation='softmax'))
            model.add(Dense(int(window*2), activation='softmax'))                            
            model.add(Dense(int(window*2), activation='tanh'))  
            model.add(Dense(int(window*1.5), activation='tanh'))
            model.add(Dense(output_size, activation='softmax'))            
            model.compile(optimizer='adagrad', loss='mean_squared_error')
            result.append([ann_name,model])
            
            return result

In [44]:
attribsSet = [['Close','Volume','High','Low']]
#attribsSet = [['Close','Volume','High','Low']]
ts_acc_best=-2.0
best=[]
epochsList=[200,300]
output_size=1
for attribIndex in range(0,len(attribsSet)):    
    closeVoldf = dfCombined[attribsSet[attribIndex]].values  
    train_percentages=[0.3,0.5,0.7]    
    window_sizes = [5,10,15,20,30,50,60,65,80,90,100,120,150]
    
    price_arr = np.array(dfCombined['Close'])
    z_arr = np.array(dfCombined['Label'])
    for train_per in train_percentages:  
        resultsANN=[]            
        for window in window_sizes:
            dataset_train = np.array(closeVoldf[:int(closeVoldf.shape[0]*train_per)])
            dataset_test = np.array(closeVoldf[int(closeVoldf.shape[0]*train_per)-window:])
            price_arr_train = np.array(price_arr[:int(price_arr.shape[0]*train_per)])
            z_arr_train = np.array(z_arr[:int(z_arr.shape[0]*train_per)])        
            price_arr_test = np.array(price_arr[int(price_arr.shape[0]*train_per)-window:])
            z_arr_test = np.array(z_arr[int(z_arr.shape[0]*train_per)-window:])

            price_arr_train = price_arr_train.reshape(-1,1)
            price_arr_test = price_arr_test.reshape(-1,1)
            min_max_scaler = MinMaxScaler(feature_range=(0,1))
            dataset_train = min_max_scaler.fit_transform(dataset_train)
            price_arr_train = min_max_scaler.fit_transform(price_arr_train)
            dataset_test = min_max_scaler.fit_transform(dataset_test)
            price_arr_test = min_max_scaler.fit_transform(price_arr_test)


            x_train, y_train, z_train = create_dataset_reg(dataset_train,price_arr_train,z_arr_train, window)
            x_test, y_test, z_test = create_dataset_reg(dataset_test,price_arr_test,z_arr_test, window)
            start = time.time()
            print('Creating models for window size ',window)
            modelList = create_models(window)
            print('Created models for window size ',window,' in ',(time.time()-start))
            for name, model in modelList:
                for epoch in epochsList:
                    print('Running model ',name,' with epoch ',epoch,attribsSet[attribIndex],window,train_per)
                    start = time.time()
                    #training the model
                    #history = model.fit(x_train, y_train, epochs=50, batch_size = 2)
                    history = model.fit(x_train, y_train, validation_split=0.15, epochs=epoch, batch_size=40, verbose=0,shuffle=False)
                    #print(model.summary())
                    y_test_pred=model.predict(x_test)
                    ts_acc = accuracy_of_direction(y_test_pred,z_test,y_test)
                    print('Completed model ',name,' with epoch ',epoch,
                          [attribsSet[attribIndex],train_per,window,name,epoch,ts_acc],
                          'Time ',(time.time()-start)
                         )
                    
                    resultsANN.append([attribsSet[attribIndex],train_per,window,name,epoch,ts_acc])
                    if(ts_acc_best <= ts_acc) :
                            ts_acc_best = ts_acc
                            best = [train_per,window,attribsSet[attribIndex],name]    
        save_results_to_csv(resultsANN,
                    ['Attributes','Train Per','Window size','ModelName','Epoch','Accuracy']
                    ,'results_ann'+str(train_per))                      

print('Best ',best,'Acc',ts_acc_best)  

Creating models for window size  5
Created models for window size  5  in  0.30549049377441406
Running model  model_1_2hidden_relu_adam  with epoch  100 ['Close', 'Volume', 'High', 'Low'] 5 0.3
Completed model  model_1_2hidden_relu_adam  with epoch  100 [['Close', 'Volume', 'High', 'Low'], 0.3, 5, 'model_1_2hidden_relu_adam', 100, 0.5283560660445082] Time  3.4768426418304443
Running model  model_1_2hidden_relu_adam  with epoch  200 ['Close', 'Volume', 'High', 'Low'] 5 0.3
Completed model  model_1_2hidden_relu_adam  with epoch  200 [['Close', 'Volume', 'High', 'Low'], 0.3, 5, 'model_1_2hidden_relu_adam', 200, 0.4946159368269921] Time  6.051752328872681
Running model  model_1_2hidden_relu_adam  with epoch  300 ['Close', 'Volume', 'High', 'Low'] 5 0.3
Completed model  model_1_2hidden_relu_adam  with epoch  300 [['Close', 'Volume', 'High', 'Low'], 0.3, 5, 'model_1_2hidden_relu_adam', 300, 0.49605168700646085] Time  9.818971157073975
Running model  model_2_2hidden_tanh_adam  with epoch  100 

Completed model  model_2_2hidden_tanh_adam  with epoch  200 [['Close', 'Volume', 'High', 'Low'], 0.3, 10, 'model_2_2hidden_tanh_adam', 200, 0.5082555635319455] Time  6.216801404953003
Running model  model_2_2hidden_tanh_adam  with epoch  300 ['Close', 'Volume', 'High', 'Low'] 10 0.3
Completed model  model_2_2hidden_tanh_adam  with epoch  300 [['Close', 'Volume', 'High', 'Low'], 0.3, 10, 'model_2_2hidden_tanh_adam', 300, 0.5154343144292893] Time  9.805509567260742
Running model  model_3_2hidden_softmax_adam  with epoch  100 ['Close', 'Volume', 'High', 'Low'] 10 0.3
Completed model  model_3_2hidden_softmax_adam  with epoch  100 [['Close', 'Volume', 'High', 'Low'], 0.3, 10, 'model_3_2hidden_softmax_adam', 100, 0.5355348169418521] Time  3.8683431148529053
Running model  model_3_2hidden_softmax_adam  with epoch  200 ['Close', 'Volume', 'High', 'Low'] 10 0.3
Completed model  model_3_2hidden_softmax_adam  with epoch  200 [['Close', 'Volume', 'High', 'Low'], 0.3, 10, 'model_3_2hidden_softmax_a

Completed model  model_3_2hidden_softmax_adam  with epoch  300 [['Close', 'Volume', 'High', 'Low'], 0.3, 15, 'model_3_2hidden_softmax_adam', 300, 0.5355348169418521] Time  11.095838785171509
Running model  model_4_2hidden_softmax_sgd  with epoch  100 ['Close', 'Volume', 'High', 'Low'] 15 0.3
Completed model  model_4_2hidden_softmax_sgd  with epoch  100 [['Close', 'Volume', 'High', 'Low'], 0.3, 15, 'model_4_2hidden_softmax_sgd', 100, 0.5355348169418521] Time  4.289254665374756
Running model  model_4_2hidden_softmax_sgd  with epoch  200 ['Close', 'Volume', 'High', 'Low'] 15 0.3
Completed model  model_4_2hidden_softmax_sgd  with epoch  200 [['Close', 'Volume', 'High', 'Low'], 0.3, 15, 'model_4_2hidden_softmax_sgd', 200, 0.5355348169418521] Time  7.939837217330933
Running model  model_4_2hidden_softmax_sgd  with epoch  300 ['Close', 'Volume', 'High', 'Low'] 15 0.3
Completed model  model_4_2hidden_softmax_sgd  with epoch  300 [['Close', 'Volume', 'High', 'Low'], 0.3, 15, 'model_4_2hidden_so

Completed model  model_5_2hidden_softmax_adagrad  with epoch  100 [['Close', 'Volume', 'High', 'Low'], 0.3, 20, 'model_5_2hidden_softmax_adagrad', 100, 0.5355348169418521] Time  4.238806486129761
Running model  model_5_2hidden_softmax_adagrad  with epoch  200 ['Close', 'Volume', 'High', 'Low'] 20 0.3
Completed model  model_5_2hidden_softmax_adagrad  with epoch  200 [['Close', 'Volume', 'High', 'Low'], 0.3, 20, 'model_5_2hidden_softmax_adagrad', 200, 0.5355348169418521] Time  7.514506816864014
Running model  model_5_2hidden_softmax_adagrad  with epoch  300 ['Close', 'Volume', 'High', 'Low'] 20 0.3
Completed model  model_5_2hidden_softmax_adagrad  with epoch  300 [['Close', 'Volume', 'High', 'Low'], 0.3, 20, 'model_5_2hidden_softmax_adagrad', 300, 0.5355348169418521] Time  11.280606031417847
Running model  model_6_4hidden_relu_adam  with epoch  100 ['Close', 'Volume', 'High', 'Low'] 20 0.3
Completed model  model_6_4hidden_relu_adam  with epoch  100 [['Close', 'Volume', 'High', 'Low'], 0.

Completed model  model_6_4hidden_relu_adam  with epoch  200 [['Close', 'Volume', 'High', 'Low'], 0.3, 30, 'model_6_4hidden_relu_adam', 200, 0.5362526920315865] Time  8.62639045715332
Running model  model_6_4hidden_relu_adam  with epoch  300 ['Close', 'Volume', 'High', 'Low'] 30 0.3
Completed model  model_6_4hidden_relu_adam  with epoch  300 [['Close', 'Volume', 'High', 'Low'], 0.3, 30, 'model_6_4hidden_relu_adam', 300, 0.5326633165829145] Time  12.039408445358276
Running model  model_7_4hidden_tanh_relu_adam  with epoch  100 ['Close', 'Volume', 'High', 'Low'] 30 0.3
Completed model  model_7_4hidden_tanh_relu_adam  with epoch  100 [['Close', 'Volume', 'High', 'Low'], 0.3, 30, 'model_7_4hidden_tanh_relu_adam', 100, 0.5362526920315865] Time  4.540095567703247
Running model  model_7_4hidden_tanh_relu_adam  with epoch  200 ['Close', 'Volume', 'High', 'Low'] 30 0.3
Completed model  model_7_4hidden_tanh_relu_adam  with epoch  200 [['Close', 'Volume', 'High', 'Low'], 0.3, 30, 'model_7_4hidden_

...

Completed model  model_7_4hidden_tanh_relu_adam  with epoch  300 [['Close', 'Volume', 'High', 'Low'], 0.7, 65, 'model_7_4hidden_tanh_relu_adam', 300, 0.46901172529313234] Time  26.836540937423706
Running model  model_8_4hidden_tanh_softmax_adagrad  with epoch  100 ['Close', 'Volume', 'High', 'Low'] 65 0.7
Completed model  model_8_4hidden_tanh_softmax_adagrad  with epoch  100 [['Close', 'Volume', 'High', 'Low'], 0.7, 65, 'model_8_4hidden_tanh_softmax_adagrad', 100, 0.5326633165829145] Time  9.292446374893188
Running model  model_8_4hidden_tanh_softmax_adagrad  with epoch  200 ['Close', 'Volume', 'High', 'Low'] 65 0.7
Completed model  model_8_4hidden_tanh_softmax_adagrad  with epoch  200 [['Close', 'Volume', 'High', 'Low'], 0.7, 65, 'model_8_4hidden_tanh_softmax_adagrad', 200, 0.5326633165829145] Time  17.818233966827393
Running model  model_8_4hidden_tanh_softmax_adagrad  with epoch  300 ['Close', 'Volume', 'High', 'Low'] 65 0.7
Completed model  model_8_4hidden_tanh_softmax_adagrad  wit

Created models for window size  90  in  0.3857765197753906
Running model  model_1_2hidden_relu_adam  with epoch  100 ['Close', 'Volume', 'High', 'Low'] 90 0.7
Completed model  model_1_2hidden_relu_adam  with epoch  100 [['Close', 'Volume', 'High', 'Low'], 0.7, 90, 'model_1_2hidden_relu_adam', 100, 0.5326633165829145] Time  9.096099376678467
Running model  model_1_2hidden_relu_adam  with epoch  200 ['Close', 'Volume', 'High', 'Low'] 90 0.7
Completed model  model_1_2hidden_relu_adam  with epoch  200 [['Close', 'Volume', 'High', 'Low'], 0.7, 90, 'model_1_2hidden_relu_adam', 200, 0.5309882747068677] Time  16.928744792938232
Running model  model_1_2hidden_relu_adam  with epoch  300 ['Close', 'Volume', 'High', 'Low'] 90 0.7
Completed model  model_1_2hidden_relu_adam  with epoch  300 [['Close', 'Volume', 'High', 'Low'], 0.7, 90, 'model_1_2hidden_relu_adam', 300, 0.5309882747068677] Time  25.533187866210938
Running model  model_2_2hidden_tanh_adam  with epoch  100 ['Close', 'Volume', 'High', '

Completed model  model_2_2hidden_tanh_adam  with epoch  100 [['Close', 'Volume', 'High', 'Low'], 0.7, 100, 'model_2_2hidden_tanh_adam', 100, 0.5393634840871022] Time  10.016324520111084
Running model  model_2_2hidden_tanh_adam  with epoch  200 ['Close', 'Volume', 'High', 'Low'] 100 0.7
Completed model  model_2_2hidden_tanh_adam  with epoch  200 [['Close', 'Volume', 'High', 'Low'], 0.7, 100, 'model_2_2hidden_tanh_adam', 200, 0.5577889447236181] Time  19.216744661331177
Running model  model_2_2hidden_tanh_adam  with epoch  300 ['Close', 'Volume', 'High', 'Low'] 100 0.7
Completed model  model_2_2hidden_tanh_adam  with epoch  300 [['Close', 'Volume', 'High', 'Low'], 0.7, 100, 'model_2_2hidden_tanh_adam', 300, 0.5494137353433836] Time  28.886720895767212
Running model  model_3_2hidden_softmax_adam  with epoch  100 ['Close', 'Volume', 'High', 'Low'] 100 0.7
Completed model  model_3_2hidden_softmax_adam  with epoch  100 [['Close', 'Volume', 'High', 'Low'], 0.7, 100, 'model_3_2hidden_softmax_a

Completed model  model_3_2hidden_softmax_adam  with epoch  200 [['Close', 'Volume', 'High', 'Low'], 0.7, 120, 'model_3_2hidden_softmax_adam', 200, 0.5309882747068677] Time  23.302353620529175
Running model  model_3_2hidden_softmax_adam  with epoch  300 ['Close', 'Volume', 'High', 'Low'] 120 0.7
Completed model  model_3_2hidden_softmax_adam  with epoch  300 [['Close', 'Volume', 'High', 'Low'], 0.7, 120, 'model_3_2hidden_softmax_adam', 300, 0.5309882747068677] Time  35.247013568878174
Running model  model_4_2hidden_softmax_sgd  with epoch  100 ['Close', 'Volume', 'High', 'Low'] 120 0.7
Completed model  model_4_2hidden_softmax_sgd  with epoch  100 [['Close', 'Volume', 'High', 'Low'], 0.7, 120, 'model_4_2hidden_softmax_sgd', 100, 0.5309882747068677] Time  11.439895153045654
Running model  model_4_2hidden_softmax_sgd  with epoch  200 ['Close', 'Volume', 'High', 'Low'] 120 0.7
Completed model  model_4_2hidden_softmax_sgd  with epoch  200 [['Close', 'Volume', 'High', 'Low'], 0.7, 120, 'model_

Completed model  model_4_2hidden_softmax_sgd  with epoch  300 [['Close', 'Volume', 'High', 'Low'], 0.7, 150, 'model_4_2hidden_softmax_sgd', 300, 0.5326633165829145] Time  41.514484167099
Running model  model_5_2hidden_softmax_adagrad  with epoch  100 ['Close', 'Volume', 'High', 'Low'] 150 0.7
Completed model  model_5_2hidden_softmax_adagrad  with epoch  100 [['Close', 'Volume', 'High', 'Low'], 0.7, 150, 'model_5_2hidden_softmax_adagrad', 100, 0.5326633165829145] Time  15.892461061477661
Running model  model_5_2hidden_softmax_adagrad  with epoch  200 ['Close', 'Volume', 'High', 'Low'] 150 0.7
Completed model  model_5_2hidden_softmax_adagrad  with epoch  200 [['Close', 'Volume', 'High', 'Low'], 0.7, 150, 'model_5_2hidden_softmax_adagrad', 200, 0.5326633165829145] Time  28.73056983947754
Running model  model_5_2hidden_softmax_adagrad  with epoch  300 ['Close', 'Volume', 'High', 'Low'] 150 0.7
Completed model  model_5_2hidden_softmax_adagrad  with epoch  300 [['Close', 'Volume', 'High', 'L

In [37]:
y_test.shape

(2476, 1)